<a href="https://colab.research.google.com/github/chicago007/test2/blob/main/5%EC%9D%BC%EC%83%81%EC%8A%B9%EC%A2%85%EB%AA%A9%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install requests konlpy
!pip install Article
!pip install pykrx
!pip install pandas_market_calendars

ERROR: Could not find a version that satisfies the requirement Article (from versions: none)
ERROR: No matching distribution found for Article


# 최근 5거래일 동안 20% 이상 상승한 종목의 1) 상승율, 시가총액 2) 뉴스 3) 검색량 증가율

In [23]:
from pykrx import stock
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
import requests
#import datetime
import re
import json
import pandas as pd
# from pykrx import stock


# 1) 최근 5거래일 20% 이상 상승한 종목의 상승률과 시가총액을 보여준다.
def get_krx_trading_range():
    krx = mcal.get_calendar('XKRX')
    today = datetime.now().date()
    start_date = today - timedelta(days=30)
    trading_days = krx.valid_days(start_date, today)
    most_recent_trading_day = trading_days[-1]
    end_date = trading_days[-5]
    return end_date.date(), most_recent_trading_day.date()

start_date, end_date = get_krx_trading_range()

print(f"시작일: {start_date}, 종료일: {end_date}")

# 주가 변동 데이터 가져오기
df = stock.get_market_price_change(start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"))

# 등락률이 20% 이상인 종목 필터링 및 내림차순 정렬
rising_stocks = df[df['등락률'] >= 20].sort_values('등락률', ascending=False)

# 시가총액 정보 가져오기
market_cap = stock.get_market_cap(end_date.strftime("%Y%m%d"))

# 결과 출력 및 종목명 리스트 생성
print("최근 5일간 20% 이상 상승한 종목:")
stock_names = []
for ticker in rising_stocks.index:
    stock_name = stock.get_market_ticker_name(ticker)
    change_rate = rising_stocks.loc[ticker, '등락률']
    # 개별 종목 시가총액 가져오기
    cap = market_cap.loc[ticker, '시가총액']
    # cap_df = stock.get_market_cap(end_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"), ticker)
    cap_billion = cap / 100_000_000  # 억원 단위로 변환
    print(f"{ticker} {stock_name} (등락률: {change_rate:.2f}%, 시가총액: {cap_billion:.2f}억원)")
    stock_names.append(stock_name)

# print("\n상승 종목명 리스트:")
# print(stock_names)


# 상승종목의 검색량 증가율을 가져온다.

# 네이버 API 정보 (발급받은 값 입력)
CLIENT_ID = "DjAgMkN04y03NqmM6SpU"
CLIENT_SECRET = "y5vuriJA8E"

# 날짜 설정
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=30)  # 최근 30일 데이터

def get_trend_data(keywords):
    url = "https://openapi.naver.com/v1/datalab/search"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET,
        "Content-Type": "application/json"
    }

    # Split keywords into chunks of 5
    chunk_size = 5
    keyword_chunks = [keywords[i:i + chunk_size] for i in range(0, len(keywords), chunk_size)]

    all_results = []
    for chunk in keyword_chunks:
        body = {
            "startDate": START_DATE.strftime("%Y-%m-%d"),
            "endDate": END_DATE.strftime("%Y-%m-%d"),
            "timeUnit": "date",
            "keywordGroups": [{"groupName": keyword, "keywords": [keyword]} for keyword in chunk]
        }

        response = requests.post(url, headers=headers, data=json.dumps(body))

        # Print the response content for debugging
        # print("API Response:", response.text)

        data = json.loads(response.text)

        # Check if 'results' key exists before accessing it
        if 'results' in data:
            all_results.extend(data["results"])  # Extend all_results with the results from this chunk
        else:
            # Handle the case where 'results' is not found
            print("Error: 'results' key not found in API response for chunk:", chunk)
            print("API Response:", response.text)  # Print the full API response for debugging
            # You might want to raise an exception here or handle the error differently
            # For now, we'll continue with the next chunk

    return all_results  # Return the combined results from all chunks

def analyze_trends(trend_data):
    df_list = []
    for item in trend_data:
        df = pd.DataFrame(item["data"])
        df["keyword"] = item["title"]
        df_list.append(df)

    # If df_list is empty, there is no data to concatenate
    # This could be due to an error in the API request or an issue with the data returned by the API.
    if not df_list:
        print("Error: No data to analyze. Check the API response and parameters.")
        return pd.DataFrame()  # Return an empty DataFrame

    df_combined = pd.concat(df_list)
    df_pivot = df_combined.pivot(index="period", columns="keyword", values="ratio")

    # 최근 7일과 이전 7일의 평균 검색량 비교
    recent_7days = df_pivot.iloc[-7:].mean()
    previous_7days = df_pivot.iloc[-14:-7].mean()
    growth_rate = (recent_7days - previous_7days) / previous_7days * 100

    return growth_rate.sort_values(ascending=False)

# 트렌드 데이터 가져오기
trend_data = get_trend_data(stock_names)

# 트렌드 분석
growth_rates = analyze_trends(trend_data)

print("\n검색량 증가율 (%):")
print(growth_rates)

# print("\n검색량이 가장 급증한 종목:")
# # Check if growth_rates is empty before accessing its index
# if not growth_rates.empty:
#     print(growth_rates.index[0])
# else:
#     print("No data available to determine the most trending stock.")

# 상승종목의 뉴스를 가져온다

# 날짜 범위 설정
end_date = datetime.now()
start_date = end_date - timedelta(days=7)

# 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret": CLIENT_SECRET
}

# 특수 문자 및 HTML 태그 제거 함수
def clean_text(text):
    text = text.replace("&amp;", "&")
    text = text.replace("quot;", '')
    text = re.sub(r'<.*?>', '', text)  # HTML 태그 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
    return text

# 각 키워드에 대해 뉴스 검색 수행
for query in stock_names:
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&sort=date&display=10"

    # API 호출
    response = requests.get(url, headers=headers)

    # 응답 확인
    if response.status_code == 200:
        news_data = response.json()
        print(f"\n최근 7일간 '{query}' 관련 뉴스:")

        count = 0
        for item in news_data["items"]:
            pub_date = datetime.strptime(item["pubDate"], "%a, %d %b %Y %H:%M:%S +0900")
            if start_date <= pub_date <= end_date:
                count += 1
                title = clean_text(item["title"])
                link = item["link"]
                print(f"{count}. {pub_date.strftime('%Y-%m-%d')} [{title}]({link})")

        if count == 0:
            print("최근 7일간 관련 뉴스가 없습니다.")
    else:
        print("네이버 API 호출 실패:", response.status_code)

/usr/local/lib/python3.11/dist-packages/pandas_market_calendars/market_calendar.py:179: UserWarning: ['break_start', 'break_end'] are discontinued, the dictionary `.discontinued_market_times` has the dates on which these were discontinued. The times as of those dates are incorrect, use .remove_time(market_time) to ignore a market_time.
  warnings.warn(


시작일: 2025-01-20, 종료일: 2025-01-24
최근 5일간 20% 이상 상승한 종목:
007660 이수페타시스 (등락률: 44.52%, 시가총액: 25867.79억원)
004870 티웨이홀딩스 (등락률: 43.77%, 시가총액: 1259.10억원)
010770 평화홀딩스 (등락률: 39.69%, 시가총액: 525.05억원)
091810 티웨이항공 (등락률: 38.38%, 시가총액: 8852.08억원)
070960 모나용평 (등락률: 34.17%, 시가총액: 2069.73억원)
090080 평화산업 (등락률: 30.74%, 시가총액: 700.55억원)
010100 한국무브넥스 (등락률: 28.91%, 시가총액: 1479.89억원)
002450 삼익악기 (등락률: 26.56%, 시가총액: 1358.87억원)
353200 대덕전자 (등락률: 26.26%, 시가총액: 9883.39억원)
000155 두산우 (등락률: 24.90%, 시가총액: 6214.50억원)
009470 삼화전기 (등락률: 23.98%, 시가총액: 2906.77억원)
014710 사조씨푸드 (등락률: 23.21%, 시가총액: 950.46억원)
033240 자화전자 (등락률: 21.43%, 시가총액: 3201.52억원)

검색량 증가율 (%):
keyword
사조씨푸드     690.320985
삼익악기      634.491219
평화산업      482.959201
평화홀딩스     313.716569
한국무브넥스    208.474756
모나용평       28.302858
이수페타시스      9.781026
티웨이항공       2.013949
대덕전자      -30.792955
자화전자      -37.090975
티웨이홀딩스    -40.044378
삼화전기      -43.630355
두산우       -52.405254
dtype: float64

최근 7일간 '이수페타시스' 관련 뉴스:
1. 2025-01-30 [LG이노텍 4분기 실적 혹한기 속 부진 목표주가 20만 

#네이버 뉴스분석 _ claude

In [20]:
import requests
import datetime
import re

# 네이버 API 정보 (발급받은 값 입력)
CLIENT_ID = "DjAgMkN04y03NqmM6SpU"
CLIENT_SECRET = "y5vuriJA8E"

# 검색할 키워드 리스트
keywords = ["이수페타시스",
    "티웨이홀딩스",
    "평화홀딩스",
    "티웨이항공",
    "제주반도체",
    "모나용평",
    "평화산업",
    "비에이치아이",
    "한국무브넥스",
    "지아이이노베이션",
    "삼익악기",
    "대덕전자",
    "태성",
    "삼화전기",
    "사조씨푸드",
    "자화전자",
    "비보존 제약"]

# 날짜 범위 설정
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=7)
print(type(start_date), type(end_date))

# # 요청 헤더 설정
# headers = {
#     "X-Naver-Client-Id": CLIENT_ID,
#     "X-Naver-Client-Secret": CLIENT_SECRET
# }

# # 특수 문자 및 HTML 태그 제거 함수
# def clean_text(text):
#     text = text.replace("&amp;", "&")
#     text = text.replace("quot;", '')
#     text = re.sub(r'<.*?>', '', text)  # HTML 태그 제거
#     text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
#     return text

# # 각 키워드에 대해 뉴스 검색 수행
# for query in keywords:
#     url = f"https://openapi.naver.com/v1/search/news.json?query={query}&sort=date&display=10"

#     # API 호출
#     response = requests.get(url, headers=headers)

#     # 응답 확인
#     if response.status_code == 200:
#         news_data = response.json()
#         print(f"\n최근 7일간 '{query}' 관련 뉴스:")

#         count = 0
#         for item in news_data["items"]:
#             pub_date = datetime.datetime.strptime(item["pubDate"], "%a, %d %b %Y %H:%M:%S +0900")
#             if start_date <= pub_date <= end_date:
#                 count += 1
#                 title = clean_text(item["title"])
#                 link = item["link"]
#                 print(f"{count}. {pub_date.strftime('%Y-%m-%d')} [{title}]({link})")

#         if count == 0:
#             print("최근 7일간 관련 뉴스가 없습니다.")
#     else:
#         print("네이버 API 호출 실패:", response.status_code)


<class 'datetime.datetime'> <class 'datetime.datetime'>


#네이버 카페

In [10]:
import requests
import json
from collections import Counter
from konlpy.tag import Okt
import re

# 네이버 API 정보
client_id = "DjAgMkN04y03NqmM6SpU"
client_secret = "y5vuriJA8E"

# 검색할 종목명
stock_name = "이수페타시스"

# 카페 검색 함수
def search_cafe(query, display=10):
    url = f"https://openapi.naver.com/v1/search/cafearticle.json?query={query}&display={display}&sort=date"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    response = requests.get(url, headers=headers)
    return json.loads(response.text)

# HTML 태그 제거 함수
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# 카페 게시글 분석 함수
def analyze_posts(cafe_items):
    okt = Okt()
    all_nouns = []

    for item in cafe_items:
        title = remove_html_tags(item['title'])
        description = remove_html_tags(item['description'])
        full_text = title + " " + description
        nouns = okt.nouns(full_text)
        all_nouns.extend(nouns)

    # 가장 많이 등장한 명사 5개 추출
    common_nouns = Counter(all_nouns).most_common(5)
    return common_nouns

# 메인 실행 코드
cafe_result = search_cafe(stock_name)
cafe_items = cafe_result['items']

print(f"{stock_name}에 대한 최근 카페 게시글 {len(cafe_items)}개:")
for idx, item in enumerate(cafe_items, 1):
    print(f"{idx}. {remove_html_tags(item['title'])}")
    print(f"   카페명: {item['cafename']}")
    print(f"   링크: {item['link']}")
    # Check if 'postdate' key exists before accessing it
    if 'postdate' in item:
        print(f"   작성일: {item['postdate']}")
    else:
        print("   작성일: 정보 없음")  # Print a message if 'postdate' is not found
    print()

common_words = analyze_posts(cafe_items)
print("카페 게시글에서 가장 많이 등장한 키워드:")
for word, count in common_words:
    print(f"{word}: {count}회")


이수페타시스에 대한 최근 카페 게시글 10개:
1. 20251024 (1매매타점확인)
   카페명: 주식 중급반
   링크: http://cafe.naver.com/voluntas0778/160
   작성일: 정보 없음

2. 24/1/24 상한가 및 급등종목
   카페명: 함께 고민하는 주식
   링크: http://cafe.naver.com/antstockworld/33672
   작성일: 정보 없음

3. 이수페타시스
   카페명: 차트,주식자료보물창고(꿈과희망있는방...
   링크: http://cafe.naver.com/purpleep4su/33792
   작성일: 정보 없음

4. 노벨의학상을 탈 것인가? 나 땜에 전세계 치과의사들 실업자? 도덕적 고민때문에 발표 못하고 있는 천기누설...
   카페명: 국민기업 관성
   링크: http://cafe.naver.com/vksekahf/46527
   작성일: 정보 없음

5. ☆2025년 민생경제 최우선으로 상법개정 +배당소득분리과세 세법개정 반드시 처리 우리증시 부스터업 벨류업...
   카페명: 한투연_한국주식투자자연합회
   링크: http://cafe.naver.com/hantuyon/91466
   작성일: 정보 없음

6. TIPS COIN 2025년 대박 프로젝트~!! 전세계 탈모 22조 한국시장 연 5000억 TIPS로 아도칠 것
   카페명: 국민기업 관성
   링크: http://cafe.naver.com/vksekahf/46526
   작성일: 정보 없음

7. 1/24/금 특징주
   카페명: Dream88의 주식연구소
   링크: http://cafe.naver.com/dream88260/177
   작성일: 정보 없음

8. ■ 우산 X NNN Pick 일주일을 돌아보는 리포트 정리(1월 4주차)
   카페명: 주식야독 (주식/공부/정보/시황/추천주...
   링크: http://cafe.naver.com/stocknightstudy/45733
   작성일: 정보 없음

9

블로그 분석

In [11]:
import requests
from collections import Counter
from konlpy.tag import Okt
import json

# 네이버 API 정보
client_id = "DjAgMkN04y03NqmM6SpU"
client_secret = "y5vuriJA8E"

# 검색할 키워드
keyword = "이수페타시스"

def search_blog(query, display=10):
    url = f"https://openapi.naver.com/v1/search/blog?query={query}&display={display}&sort=date"
    # url = f"https://openapi.naver.com/v1/search/query={query}&display={display}&sort=date"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    response = requests.get(url, headers=headers)
    return json.loads(response.text)

def analyze_blog_posts(blog_items):
    okt = Okt()
    all_nouns = []

    for item in blog_items:
        title = item['title']
        description = item['description']
        full_text = title + " " + description
        nouns = okt.nouns(full_text)
        all_nouns.extend(nouns)

    common_nouns = Counter(all_nouns).most_common(10)
    return common_nouns

# 메인 실행 코드
blog_result = search_blog(keyword)
blog_items = blog_result['items']

print(f"{keyword}에 대한 최근 블로그 포스트 {len(blog_items)}개:")
for idx, item in enumerate(blog_items, 1):
    print(f"{idx}. {item['title']}")
    print(f"   블로그명: {item['bloggername']}")
    print(f"   링크: {item['link']}")
    print(f"   작성일: {item['postdate']}")
    print()

common_words = analyze_blog_posts(blog_items)
print("블로그 포스트에서 가장 많이 등장한 키워드:")
for word, count in common_words:
    print(f"{word}: {count}회")


이수페타시스에 대한 최근 블로그 포스트 10개:
1. [매매일지] 경제적 자유 단타일지
   블로그명: voluntas07님의 블로그
   링크: https://blog.naver.com/voluntas07/223741772706
   작성일: 20250129

2. [매매연구] 단타 트레이딩 매매루틴 만들기, 시초가 매매후... 
   블로그명: 상상대로 트레이더
   링크: https://blog.naver.com/kwsuhan/223741711475
   작성일: 20250129

3. 네프콘 특강 : 수급 차트 패턴
   블로그명: 수급 퀀트의 마법사
   링크: https://blog.naver.com/nomadrid/223741697013
   작성일: 20250129

4. 딥시크 관련주, AI 시장을 뒤흔든 기회
   블로그명: 머니포스 전략실
   링크: https://blog.naver.com/jong8783/223741680328
   작성일: 20250129

5. 1월결산
   블로그명: 창형의생각
   링크: https://blog.naver.com/toxic0205/223741644988
   작성일: 20250129

6. 29일 미주식[ ASML/  Advantest 도 컨센서스를 상회하는... 
   블로그명: 밝은내일주식
   링크: https://blog.naver.com/coo2777/223741638503
   작성일: 20250129

7. [2025년 2월 주식시장 및 주도테마주 전망]
   블로그명: 더와이즈스탁
   링크: https://blog.naver.com/gotyou99/223741531196
   작성일: 20250129

8. 스승님 과제 3: 1월 오답노트
   블로그명: 스따
   링크: https://blog.naver.com/strattonoakmont/223741510146
   작성일: 20250129

9. 딥시크 국내주식 관련주 8종목. 열심히 조사해봤습니다.
   블로그명: 명토르의 국

네이버 검색

In [37]:
import requests
from collections import Counter
from konlpy.tag import Okt
import json
import re

# 네이버 API 정보
client_id = "DjAgMkN04y03NqmM6SpU"
client_secret = "y5vuriJA8E"

# 검색할 키워드
keyword = "평화홀딩스"

def search_web(query, display=100):
    url = f"https://openapi.naver.com/v1/search/webkr?query={query}&display={display}&sort=sim"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    response = requests.get(url, headers=headers)
    return json.loads(response.text)

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def analyze_search_results(items):
    okt = Okt()
    all_nouns = []

    for item in items:
        title = remove_html_tags(item['title'])
        description = remove_html_tags(item['description'])
        full_text = title + " " + description
        nouns = okt.nouns(full_text)
        all_nouns.extend(nouns)

    common_nouns = Counter(all_nouns).most_common(10)
    return common_nouns

# 메인 실행 코드
search_result = search_web(keyword)
search_items = search_result['items']

print(f"{keyword}에 대한 웹 검색 결과 {len(search_items)}개:")
for idx, item in enumerate(search_items, 1):
    print(f"{idx}. {remove_html_tags(item['title'])}")
    print(f"   링크: {item['link']}")
    print(f"   설명: {remove_html_tags(item['description'][:100])}...")
    print()

common_words = analyze_search_results(search_items)
print("검색 결과에서 가장 많이 등장한 키워드:")
for word, count in common_words:
    print(f"{word}: {count}회")


평화홀딩스에 대한 웹 검색 결과 75개:
1. 평화홀딩스(주)
   링크: http://www.ph.co.kr/
   설명: 자동차부품 제조 전문업체, 방진고무, 호스, 씰링제품 개발 및 제조 등 안내...

2. 평화홀딩스(주) 2025년 기업정보 | 직원수, 근무환경, 복리후생 등
   링크: https://m.saramin.co.kr/job-search/company-info-view?csn=clgrMk5DWHBqQTNyS1g5Mnk1NUIzZz09
   설명: 평화홀딩스(주) 기업소개 - 업력 : 51년차, 기업형태 : -, 업종 : 그 외 자동차용 신품 부품 제조업 | 평화홀딩스(주)의 직원수, 연봉, 채용, ...

3. 평화홀딩스㈜ 2025년 기업정보 - 직원수, 근무환경, 복리후생 등
   링크: https://www.jobkorea.co.kr/company/1468190
   설명: 평화홀딩스㈜의 최신 소식 및 기업문화, 근무환경, 고용현황, 직원수 등의 기업정보를 확인해보세요....

4. 평화홀딩스(주) - 인크루트
   링크: https://www.incruit.com/company/1110322
   설명: 평화홀딩스의 최신 기업정보, 채용, 연봉 등 취업에 필요한 정보를 인크루트에서 확인하세요....

5. 평화홀딩스 - 위키백과 한국어
   링크: https://ko.wikipedia.org/wiki/%ED%8F%89%ED%99%94%ED%99%80%EB%94%A9%EC%8A%A4
   설명: 평화홀딩스 (Pyung Hwa Holdings Co. Ltd.)는 대한민국의 기업이다. 본사는 대구시 달성군 논공읍 논공로 597에 위치해 있으며 대표이사는 김종석이다...

6. 평화홀딩스: 3,590 ▼-375 (-9.46%)
   링크: https://finance.daum.net/quotes/A010770
   설명: 동사는 다른 회사의 주식을 소유함으로써 그 회사를 지배하는 것을 목적으로 하는 지주회사이며 지주회사의 영업수익은 용역수익

#최근 5일동안 주가가 20% 상승한 종목

In [1]:
!pip install pykrx
!pip install pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 11.8 MB/s eta 0:00:00


In [13]:
from pykrx import stock
from datetime import datetime, timedelta
import pandas_market_calendars as mcal

def get_krx_trading_range():
    krx = mcal.get_calendar('XKRX')
    today = datetime.now().date()
    start_date = today - timedelta(days=30)
    trading_days = krx.valid_days(start_date, today)
    most_recent_trading_day = trading_days[-1]
    end_date = trading_days[-5]
    return end_date.date(), most_recent_trading_day.date()

start_date, end_date = get_krx_trading_range()

print(f"시작일: {start_date}, 종료일: {end_date}")

# 주가 변동 데이터 가져오기
df = stock.get_market_price_change(start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"))

# 등락률이 20% 이상인 종목 필터링 및 내림차순 정렬
rising_stocks = df[df['등락률'] >= 20].sort_values('등락률', ascending=False)

# 시가총액 정보 가져오기
market_cap = stock.get_market_cap(end_date.strftime("%Y%m%d"))
# print(market_cap)
# 결과 출력 및 종목명 리스트 생성
print("최근 5일간 20% 이상 상승한 종목:")
stock_names = []
for ticker in rising_stocks.index:
    stock_name = stock.get_market_ticker_name(ticker)
    change_rate = rising_stocks.loc[ticker, '등락률']
    # 개별 종목 시가총액 가져오기
    cap = market_cap.loc[ticker, '시가총액']
    # cap_df = stock.get_market_cap(end_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"), ticker)
    cap_billion = cap / 100_000_000  # 억원 단위로 변환
    print(f"{ticker} {stock_name} (등락률: {change_rate:.2f}%, 시가총액: {cap_billion:.2f}억원)")
    stock_names.append(stock_name)

print("\n상승 종목명 리스트:")
print(stock_names)


/usr/local/lib/python3.11/dist-packages/pandas_market_calendars/market_calendar.py:179: UserWarning: ['break_start', 'break_end'] are discontinued, the dictionary `.discontinued_market_times` has the dates on which these were discontinued. The times as of those dates are incorrect, use .remove_time(market_time) to ignore a market_time.
  warnings.warn(


시작일: 2025-01-20, 종료일: 2025-01-24
최근 5일간 20% 이상 상승한 종목:
007660 이수페타시스 (등락률: 44.52%, 시가총액: 25867.79억원)
004870 티웨이홀딩스 (등락률: 43.77%, 시가총액: 1259.10억원)
010770 평화홀딩스 (등락률: 39.69%, 시가총액: 525.05억원)
091810 티웨이항공 (등락률: 38.38%, 시가총액: 8852.08억원)
070960 모나용평 (등락률: 34.17%, 시가총액: 2069.73억원)
090080 평화산업 (등락률: 30.74%, 시가총액: 700.55억원)
010100 한국무브넥스 (등락률: 28.91%, 시가총액: 1479.89억원)
002450 삼익악기 (등락률: 26.56%, 시가총액: 1358.87억원)
353200 대덕전자 (등락률: 26.26%, 시가총액: 9883.39억원)
000155 두산우 (등락률: 24.90%, 시가총액: 6214.50억원)
009470 삼화전기 (등락률: 23.98%, 시가총액: 2906.77억원)
014710 사조씨푸드 (등락률: 23.21%, 시가총액: 950.46억원)
033240 자화전자 (등락률: 21.43%, 시가총액: 3201.52억원)

상승 종목명 리스트:
['이수페타시스', '티웨이홀딩스', '평화홀딩스', '티웨이항공', '모나용평', '평화산업', '한국무브넥스', '삼익악기', '대덕전자', '두산우', '삼화전기', '사조씨푸드', '자화전자']


#네이버 뉴스, 카페, 블로그, 검색에서 키워드 검색

In [7]:
import requests
from datetime import datetime, timedelta
from collections import Counter
from konlpy.tag import Okt

# 네이버 API 정보
CLIENT_ID = "DjAgMkN04y03NqmM6SpU"
CLIENT_SECRET = "y5vuriJA8E"

# 검색어 및 날짜 설정
QUERY = "이수페타시스"
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=7)

def search_naver(api_type):
    url = f"https://openapi.naver.com/v1/search/{api_type}.json"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET
    }
    params = {
        "query": QUERY,
        "display": 100,
        "start": 1,
        "sort": "date"
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json().get('items', [])

def extract_keywords(text):
    okt = Okt()
    nouns = okt.nouns(text)
    return [noun for noun in nouns if len(noun) > 1]

def main():
    all_keywords = []
    # api_types = ['news', 'webkr', 'cafearticle', 'blog']
    api_types = ['news', 'cafearticle', 'blog']

    for api_type in api_types:
        items = search_naver(api_type)
        for item in items:
            # Check if 'pubDate' key exists before accessing it
            if 'pubDate' in item:
                date = datetime.strptime(item['pubDate'], "%a, %d %b %Y %H:%M:%S +0900")
                if START_DATE <= date <= END_DATE:
                    title = item.get('title', '')
                    description = item.get('description', '')
                    all_keywords.extend(extract_keywords(title + " " + description))
            # If 'pubDate' is not found, try 'postdate' (for cafe articles)
            elif 'postdate' in item:
                try:
                    date = datetime.strptime(str(item['postdate']), "%Y%m%d")  # Assuming 'postdate' is in YYYYMMDD format
                    if START_DATE <= date <= END_DATE:
                        title = item.get('title', '')
                        description = item.get('description', '')
                        all_keywords.extend(extract_keywords(title + " " + description))
                except ValueError:
                    print(f"Warning: Invalid date format for 'postdate': {item['postdate']}")
            else:
                print(f"Warning: No date information found for item: {item}")  # Handle cases where no date key is found

    keyword_counts = Counter(all_keywords)
    top_keywords = keyword_counts.most_common(10)

    print(f"'{QUERY}'와 관련된 최근 일주일간 가장 많이 언급된 키워드 Top 10:")
    for keyword, count in top_keywords:
        print(f"{keyword}: {count}회")

if __name__ == "__main__":
    main()

'이수페타시스'와 관련된 최근 일주일간 가장 많이 언급된 키워드 Top 10:
시스: 411회
페타: 404회
이오: 165회
인수: 154회
철회: 122회
주가: 114회
급등: 73회
거래: 65회
기업: 58회
주식: 55회


#네이버 트렌드에서 시총 300위 종목의 검색량 증가율

In [3]:
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
# from pykrx import stock


# 네이버 API 정보
CLIENT_ID = "DjAgMkN04y03NqmM6SpU"
CLIENT_SECRET = "y5vuriJA8E"

# 날짜 설정
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=10)  # 최근 30일 데이터

def get_trend_data(keywords):
    url = "https://openapi.naver.com/v1/datalab/search"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET,
        "Content-Type": "application/json"
    }

    # Split keywords into chunks of 5
    chunk_size = 5
    keyword_chunks = [keywords[i:i + chunk_size] for i in range(0, len(keywords), chunk_size)]

    all_results = []
    for chunk in keyword_chunks:
        body = {
            "startDate": START_DATE.strftime("%Y-%m-%d"),
            "endDate": END_DATE.strftime("%Y-%m-%d"),
            "timeUnit": "date",
            "keywordGroups": [{"groupName": keyword, "keywords": [keyword]} for keyword in chunk]
        }

        response = requests.post(url, headers=headers, data=json.dumps(body))

        # Print the response content for debugging
        # print("API Response:", response.text)

        data = json.loads(response.text)

        # Check if 'results' key exists before accessing it
        if 'results' in data:
            all_results.extend(data["results"])  # Extend all_results with the results from this chunk
        else:
            # Handle the case where 'results' is not found
            print("Error: 'results' key not found in API response for chunk:", chunk)
            print("API Response:", response.text)  # Print the full API response for debugging
            # You might want to raise an exception here or handle the error differently
            # For now, we'll continue with the next chunk

    return all_results  # Return the combined results from all chunks


def analyze_trends(trend_data):
    df_list = []
    for item in trend_data:
        df = pd.DataFrame(item["data"])
        df["keyword"] = item["title"]
        df_list.append(df)

    # If df_list is empty, there is no data to concatenate
    # This could be due to an error in the API request or an issue with the data returned by the API.
    if not df_list:
        print("Error: No data to analyze. Check the API response and parameters.")
        return pd.DataFrame()  # Return an empty DataFrame

    df_combined = pd.concat(df_list)
    df_pivot = df_combined.pivot(index="period", columns="keyword", values="ratio")

    # 최근 7일과 이전 7일의 평균 검색량 비교
    recent_7days = df_pivot.iloc[-7:].mean()
    previous_7days = df_pivot.iloc[-14:-7].mean()
    growth_rate = (recent_7days - previous_7days) / previous_7days * 100

    return growth_rate.sort_values(ascending=False)

# def get_stock_names():
#     tickers = stock.get_market_ticker_list(market="ALL")
#     stock_names = [stock.get_market_ticker_name(ticker) for ticker in tickers]
#     # i += 1
#     return stock_names

# stock_list = get_stock_names()

# 주식 종목 리스트 (예시)
# stock_list = ["삼성전자", "평화홀딩스", "네이버", "카카오", "현대차", "LG화학"]
stock_list = [
    "이수페타시스",
    "티웨이홀딩스",
    "평화홀딩스",
    "티웨이항공",
    "제주반도체",
    "모나용평",
    "평화산업",
    "비에이치아이",
    "한국무브넥스",
    "지아이이노베이션",
    "삼익악기",
    "대덕전자",
    "태성",
    "삼화전기",
    "사조씨푸드",
    "자화전자",
    "비보존 제약"]

# stock_list = ['삼성전자', 'SK하이닉스', 'LG에너지솔루션', '삼성바이오로직스', '현대차', '기아', '셀트리온', '삼성전자우', 'KB금융',
#  'NAVER', 'HD현대중공업', '신한지주', '현대모비스', 'POSCO홀딩스', '메리츠금융지주', '삼성물산', '알테오젠', 'SK이노베이션',
#  '한화에어로스페이스', '고려아연', '한화오션', '하나금융지주', 'LG화학', '크래프톤', 'HMM', '삼성생명', '삼성화재',
#  'HD한국조선해양', 'HD현대일렉트릭', '카카오', '두산에너빌리티', '삼성SDI', 'LG전자', '한국전력', 'SK스퀘어', 'KT&G',
#  '에코프로비엠', '기업은행', 'KT', 'SK텔레콤', '우리금융지주', '한미반도체', 'LG', '삼성중공업', '포스코퓨처엠',
#  '현대글로비스', 'SK', '유한양행', 'HLB', '카카오뱅크', '삼성전기', '하이브', '대한항공', '삼성에스디에스', 'SK바이오팜',
#  '에코프로', 'HD현대마린솔루션', '포스코인터내셔널', '아모레퍼시픽', 'LS ELECTRIC', 'S-Oil', 'HD현대', 'DB손해보험',
#  '현대로템', 'SKC', '현대차2우B', '코웨이', '두산', '한진칼', '삼양식품', '두산밥캣', '한국항공우주', 'LIG넥스원',
#  '맥쿼리인프라', 'HD현대미포', '에코프로머티', '한화시스템', '한국타이어앤테크놀로지', '미래에셋증권', '레인보우로보틱스',
#  '삼성카드', '효성중공업', 'NH투자증권', 'LG생활건강', 'LG디스플레이', '리가켐바이오', 'LG유플러스', '한국금융지주',
#  '두산로보틱스', 'LS', '삼천당제약', '삼성증권', 'SK바이오사이언스', '오리온', 'BNK금융지주', '현대오토에버',
#  '엔씨소프트', 'JB금융지주', '현대차우', '넷마블', 'GS', 'CJ제일제당', '강원랜드', 'LG이노텍', '클래시스', '현대건설',
#  '삼성E&A', '카카오페이', '한화솔루션', '시프트업', '리노공업', '한국가스공사', '키움증권', '엘앤에프', '한미약품',
#  '현대제철', '휴젤', '금호석유', '포스코DX', 'CJ', '한온시스템', '엔켐', 'HPSP', 'JYP Ent.', '한전기술',
#  '이수페타시스', '대한전선', '파마리서치', '신성델타테크', '롯데케미칼', '휠라홀딩스', 'F&F', '동서', '한화',
#  '코오롱티슈진', '에스원', '펩트론', '산일전기', '셀트리온제약', '롯데지주', 'KCC', '아시아나항공', '한올바이오파마',
#  '현대해상', '한화생명', '한전KPS', '농심', '현대엘리베이', '루닛', 'HL만도', 'SK가스', '더존비즈온', '에스엠',
#  '아모레G', '제일기획', '이오테크닉스', '실리콘투', '한화엔진', '한미사이언스', '영원무역', '코스맥스', '테크윙',
#  'CJ대한통운', '펄어비스', 'BGF리테일', '에이피알', '이마트', '팬오션', '에스티팜', '씨에스윈드', '일진전기', '한화비전',
#  '파크시스템스', '코스모신소재', '에이비엘바이오', 'SK아이이테크놀로지', '녹십자', '대주전자재료', '코리안리', '한국앤컴퍼니',
#  '대웅제약', '주성엔지니어링', '호텔신라', 'ISC', 'DGB금융지주', 'OCI홀딩스', '롯데쇼핑', '풍산', '오뚜기', 'GS건설',
#  '케어젠', 'DB하이텍', 'HD현대인프라코어', '보로노이', '에스엘', '에스디바이오센서', 'LS에코에너지', '한국콜마',
#  '대우건설', '서진시스템', '금호타이어', '솔브레인', '위메이드', '동원산업', '제이앤티씨', '하이트진로', '카카오게임즈',
#  'HD현대건설기계', 'SK리츠', '경동나비엔', '신세계', 'HLB생명과학', 'GS리테일', '씨젠', 'DL이앤씨', '신영증권',
#  '피에스케이홀딩스', 'DN오토모티브', '대웅', '동진쎄미켐', '브이티', '종근당', '네이처셀', 'CJ ENM', '원익IPS',
#  '세방전지', '금양', '스튜디오드래곤', 'HDC현대산업개발', '현대백화점', '동원시스템즈', 'LG화학우', '영원무역홀딩스',
#  '한샘', '두산퓨얼셀', '한솔케미칼', '현대위아', '고영', '더블유게임즈', '한일시멘트', 'DI동일', '롯데에너지머티리얼즈',
#  '오스코텍', '이수스페셜티케미컬', '롯데정밀화학', 'HK이노엔', '롯데렌탈', '효성티앤씨', '대덕전자', 'ESR켄달스퀘어리츠',
#  '제룡전기', '하이젠알앤엠', 'LX인터내셔널', '솔루엠', '롯데웰푸드', '메지온', '카페24', '롯데칠성', '와이씨', 'LX세미콘',
#  'SK네트웍스', '미래에셋생명', '오리온홀딩스', '와이지엔터테인먼트', 'HD현대마린엔진', 'LS머트리얼즈', '피엔티', 'SOOP',
#  '파라다이스', 'HLB테라퓨틱스', 'KB발해인프라', '유진테크', '프레스티지바이오파마', '국일제지', '보령', '롯데리츠',
#  '티웨이항공', '세아제강지주', '메디톡스', '하나투어', '넥슨게임즈', '티씨케이', 'CJ CGV', '디어유', '삼성화재우',
#  '태성', '성광벤드', '한국단자', '대신증권', 'HLB제약', '나노신소재', '미원상사', '율촌화학', '엠로', '쿠쿠홀딩스',
#  '다우기술', 'HS효성첨단소재', '안랩', '중앙첨단소재', '레이크머티리얼즈']

# 트렌드 데이터 가져오기
trend_data = get_trend_data(stock_list)

# 트렌드 분석
growth_rates = analyze_trends(trend_data)

print("검색량 증가율 (%):")
print(growth_rates)

print("\n검색량이 가장 급증한 종목:")
# Check if growth_rates is empty before accessing its index
if not growth_rates.empty:
    print(growth_rates.index[0])
else:
    print("No data available to determine the most trending stock.")

검색량 증가율 (%):
keyword
평화홀딩스       1753.969947
평화산업         992.523966
사조씨푸드        783.293926
삼익악기         677.185672
한국무브넥스       252.407270
자화전자          63.566857
태성            47.318620
이수페타시스        37.259841
비에이치아이        29.543666
티웨이홀딩스        27.342186
모나용평          26.837296
티웨이항공         10.433539
지아이이노베이션       6.978665
삼화전기         -32.897142
대덕전자         -34.414192
비보존 제약       -46.476896
제주반도체        -61.169202
dtype: float64

검색량이 가장 급증한 종목:
평화홀딩스


In [52]:
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 16.4 MB/s eta 0:00:00


In [53]:
from pykrx import stock

def get_stock_names():
    tickers = stock.get_market_ticker_list(market="ALL")
    stock_names = [stock.get_market_ticker_name(ticker) for ticker in tickers]
    i += 1
    return stock_names

stock_names = get_stock_names()
print(stock_names)

['3S', 'AJ네트웍스', 'AK홀딩스', 'APS', 'AP시스템', 'AP위성', 'AP헬스케어', 'BF랩스', 'BGF', 'BGF리테일', 'BGF에코머티리얼즈', 'BNK금융지주', 'BYC', 'BYC우', 'CBI', 'CG인바이츠', 'CJ', 'CJ CGV', 'CJ ENM', 'CJ 바이오사이언스', 'CJ4우(전환)', 'CJ대한통운', 'CJ씨푸드', 'CJ씨푸드1우', 'CJ우', 'CJ제일제당', 'CJ제일제당 우', 'CJ프레시웨이', 'CMG제약', 'CNH', 'CNT85', 'CR홀딩스', 'CS', 'CSA 코스믹', 'CS홀딩스', 'DB', 'DB금융스팩11호', 'DB금융스팩12호', 'DB금융투자', 'DB손해보험', 'DB하이텍', 'DGB금융지주', 'DGI', 'DGP', 'DH오토리드', 'DH오토웨어', 'DI동일', 'DL', 'DL우', 'DL이앤씨', 'DL이앤씨2우(전환)', 'DL이앤씨우', 'DMS', 'DN오토모티브', 'DRB동일', 'DSC인베스트먼트', 'DSR', 'DSR제강', 'DS단석', 'DXVX', 'E1', 'EDGC', 'EG', 'EMB', 'ESR켄달스퀘어리츠', 'ES큐브', 'F&F', 'F&F홀딩스', 'FSN', 'GH신소재', 'GKL', 'GRT', 'GS', 'GST', 'GS건설', 'GS글로벌', 'GS리테일', 'GS우', 'GS피앤엘', 'HB솔루션', 'HB인베스트먼트', 'HB테크놀러지', 'HDC', 'HDC랩스', 'HDC현대EP', 'HDC현대산업개발', 'HD한국조선해양', 'HD현대', 'HD현대건설기계', 'HD현대마린솔루션', 'HD현대마린엔진', 'HD현대미포', 'HD현대에너지솔루션', 'HD현대인프라코어', 'HD현대일렉트릭', 'HD현대중공업', 'HJ중공업', 'HK이노엔', 'HL D&I', 'HLB', 'HLB글로벌', 'HLB바이오스텝', 'HLB사이언스', 'HLB생명과학', 'HLB이노베이션', 'HLB제넥스', 'H

In [70]:
from pykrx import stock
import pandas as pd

# def get_top_300_stocks():
#     # 현재 날짜 설정 (2025년 1월 29일)
#     date = "20250129"

#     # KOSPI와 KOSDAQ 종목 코드 가져오기
#     kospi_tickers = stock.get_market_ticker_list(market="KOSPI")
#     # kosdaq_tickers = stock.get_market_ticker_list(market="KOSDAQ")
#     all_tickers = kospi_tickers #+ kosdaq_tickers

#     # 시가총액 정보 가져오기
#     market_caps = stock.get_market_cap_by_ticker(date, market="ALL")

#     # 종목명, 종목코드, 시가총액 정보를 DataFrame으로 변환
#     df = pd.DataFrame({
#         'name': [stock.get_market_ticker_name(ticker) for ticker in all_tickers],
#         'ticker': all_tickers,
#         'market_cap': [market_caps.loc[ticker, '시가총액'] if ticker in market_caps.index else 0 for ticker in all_tickers]
#     })

#     # 시가총액 기준으로 정렬하고 상위 300개 선택
#     top_300 = df.sort_values('market_cap', ascending=True).head(300)

#     # 종목명만 리스트로 저장
#     top_300_names = top_300['name'].tolist()

#     return top_300_names

# # 함수 실행 및 결과 저장
# top_300_stocks = get_top_300_stocks()
# print(top_300_stocks)

# df = stock.get_market_cap("20200625")
# print(df.head(30))


# from pykrx import stock
# import pandas as pd

# 시가총액 데이터 가져오기
df = stock.get_market_cap("20250124")

# 상위 300개 종목 선택
top_300 = df.head(300)

# print(top_300)

# 종목코드를 인덱스에서 컬럼으로 이동
top_300 = top_300.reset_index()

# 종목명 가져오기
top_300['종목명'] = top_300['티커'].apply(lambda x: stock.get_market_ticker_name(x))

# 종목명을 리스트로 저장
top_300_names = top_300['종목명'].tolist()

# 결과 출력
print("시가총액 상위 300개 종목:")
print(top_300_names)


시가총액 상위 300개 종목:
['삼성전자', 'SK하이닉스', 'LG에너지솔루션', '삼성바이오로직스', '현대차', '기아', '셀트리온', '삼성전자우', 'KB금융', 'NAVER', 'HD현대중공업', '신한지주', '현대모비스', 'POSCO홀딩스', '메리츠금융지주', '삼성물산', '알테오젠', 'SK이노베이션', '한화에어로스페이스', '고려아연', '한화오션', '하나금융지주', 'LG화학', '크래프톤', 'HMM', '삼성생명', '삼성화재', 'HD한국조선해양', 'HD현대일렉트릭', '카카오', '두산에너빌리티', '삼성SDI', 'LG전자', '한국전력', 'SK스퀘어', 'KT&G', '에코프로비엠', '기업은행', 'KT', 'SK텔레콤', '우리금융지주', '한미반도체', 'LG', '삼성중공업', '포스코퓨처엠', '현대글로비스', 'SK', '유한양행', 'HLB', '카카오뱅크', '삼성전기', '하이브', '대한항공', '삼성에스디에스', 'SK바이오팜', '에코프로', 'HD현대마린솔루션', '포스코인터내셔널', '아모레퍼시픽', 'LS ELECTRIC', 'S-Oil', 'HD현대', 'DB손해보험', '현대로템', 'SKC', '현대차2우B', '코웨이', '두산', '한진칼', '삼양식품', '두산밥캣', '한국항공우주', 'LIG넥스원', '맥쿼리인프라', 'HD현대미포', '에코프로머티', '한화시스템', '한국타이어앤테크놀로지', '미래에셋증권', '레인보우로보틱스', '삼성카드', '효성중공업', 'NH투자증권', 'LG생활건강', 'LG디스플레이', '리가켐바이오', 'LG유플러스', '한국금융지주', '두산로보틱스', 'LS', '삼천당제약', '삼성증권', 'SK바이오사이언스', '오리온', 'BNK금융지주', '현대오토에버', '엔씨소프트', 'JB금융지주', '현대차우', '넷마블', 'GS', 'CJ제일제당', '강원랜드', 'LG이노텍', '클래시스', '현대건설', '삼성E&A', '카카오페이

#구글트렌드

In [94]:
# !pip install pytrends
# !pip install --upgrade pytrends
# !pip install --upgrade pytrends requests urllib3
!pip install pytrends pandas requests urllib3


In [113]:
from pytrends.request import TrendReq
import pandas as pd

# Google Trends에 연결 (한국어 설정)
pytrends = TrendReq(hl='ko-KR', tz=540)

# 키워드 리스트
kw_list = ['삼성전자', 'SK하이닉스', 'LG에너지솔루션', '삼성바이오로직스', '현대차']

# 대한민국 데이터와 지난 48시간 동안의 데이터 요청
pytrends.build_payload(kw_list,
                       cat=0,
                       timeframe='now 7-d',
                       geo='KR',
                       gprop='')

# 시간에 따른 관심도 데이터 검색
iot = pytrends.interest_over_time()
iot.reset_index(inplace=True)
print(iot)


                   date  삼성전자  SK하이닉스  LG에너지솔루션  삼성바이오로직스  현대차  isPartial
0   2025-01-22 06:00:00    65      10         4         6   17      False
1   2025-01-22 07:00:00    58       9         4         6   18      False
2   2025-01-22 08:00:00    53      15         3         4   16      False
3   2025-01-22 09:00:00    41       9         3         0   12      False
4   2025-01-22 10:00:00    40      11         0         2   12      False
..                  ...   ...     ...       ...       ...  ...        ...
164 2025-01-29 02:00:00    32       5         0         4    8      False
165 2025-01-29 03:00:00    35       4         0         3    6      False
166 2025-01-29 04:00:00    34       7         0         3   10      False
167 2025-01-29 05:00:00    38       8         0         5   11      False
168 2025-01-29 06:00:00    39       6         0         0    9       True

[169 rows x 7 columns]


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


In [118]:
from pytrends.request import TrendReq
import pandas as pd

# Google Trends에 연결 (한국어 설정)
pytrends = TrendReq(hl='ko-KR', tz=540)

# 키워드 리스트
kw_list = ['삼성전자', 'SK하이닉스', 'LG에너지솔루션', '삼성바이오로직스', '현대차']

# 대한민국 데이터와 지난 7일 동안의 데이터 요청
pytrends.build_payload(kw_list,
                       cat=0,
                       timeframe='now 7-d',
                       geo='KR',
                       gprop='')

# 시간에 따른 관심도 데이터 검색
with pd.option_context('future.no_silent_downcasting', True):
    iot = pytrends.interest_over_time()
    iot = iot.infer_objects(copy=False)

# 'isPartial' 열 제거
iot = iot.drop(columns=['isPartial'])

# 각 키워드별 최대값 계산
max_values = iot.max()

# 결과 출력 (내림차순 정렬)
print(max_values.sort_values(ascending=False))


삼성전자        100
SK하이닉스       44
현대차          28
삼성바이오로직스     11
LG에너지솔루션      7
dtype: int64


In [123]:
from pytrends.request import TrendReq
import pandas as pd
import time

def get_trends_data_in_batches(keywords, base_keyword, pytrends):
    results = pd.DataFrame()

    for i in range(0, len(keywords), 4):
        batch = keywords[i:i+4]
        if base_keyword not in batch:
            batch = [base_keyword] + batch

        try:
            with pd.option_context('future.no_silent_downcasting', True):
                pytrends.build_payload(batch,
                                     cat=0,
                                     timeframe='now 7-d',
                                     geo='KR',
                                     gprop='')

                data = pytrends.interest_over_time()
                if not data.empty:
                    data = data.drop(columns=['isPartial'])
                    data = data.infer_objects(copy=False)

                    for col in data.columns:
                        if col != base_keyword:
                            results[col] = data[col] / data[base_keyword] * 100

            time.sleep(2)

        except Exception as e:
            print(f"Error processing batch {batch}: {str(e)}")
            time.sleep(5)

    results[base_keyword] = 100
    return results

# Google Trends에 연결
pytrends = TrendReq(hl='ko-KR', tz=540)

# 키워드 리스트
kw_list = ['삼성전자', 'SK하이닉스', 'LG에너지솔루션', '삼성바이오로직스', '현대차', '기아', '셀트리온', '삼성전자우', 'KB금융',
 'NAVER', 'HD현대중공업', '신한지주', '현대모비스', 'POSCO홀딩스', '메리츠금융지주', '삼성물산', '알테오젠', 'SK이노베이션',
 '한화에어로스페이스', '고려아연', '한화오션', '하나금융지주', 'LG화학', '크래프톤', 'HMM', '삼성생명', '삼성화재',
 'HD한국조선해양', 'HD현대일렉트릭', '카카오', '두산에너빌리티', '삼성SDI', 'LG전자', '한국전력', 'SK스퀘어', 'KT&G',
 '에코프로비엠', '기업은행', 'KT', 'SK텔레콤', '우리금융지주', '한미반도체', 'LG', '삼성중공업', '포스코퓨처엠',
 '현대글로비스', 'SK', '유한양행', 'HLB', '카카오뱅크', '삼성전기', '하이브', '대한항공', '삼성에스디에스', 'SK바이오팜',
 '에코프로', 'HD현대마린솔루션', '포스코인터내셔널', '아모레퍼시픽', 'LS ELECTRIC', 'S-Oil', 'HD현대', 'DB손해보험',
 '현대로템', 'SKC', '현대차2우B', '코웨이', '두산', '한진칼', '삼양식품', '두산밥캣', '한국항공우주', 'LIG넥스원',
 '맥쿼리인프라', 'HD현대미포', '에코프로머티', '한화시스템', '한국타이어앤테크놀로지', '미래에셋증권', '레인보우로보틱스',
 '삼성카드', '효성중공업', 'NH투자증권', 'LG생활건강', 'LG디스플레이', '리가켐바이오', 'LG유플러스', '한국금융지주',
 '두산로보틱스', 'LS', '삼천당제약', '삼성증권', 'SK바이오사이언스', '오리온', 'BNK금융지주', '현대오토에버',
 '엔씨소프트', 'JB금융지주', '현대차우', '넷마블', 'GS', 'CJ제일제당', '강원랜드', 'LG이노텍', '클래시스', '현대건설',
 '삼성E&A', '카카오페이', '한화솔루션', '시프트업', '리노공업', '한국가스공사', '키움증권', '엘앤에프', '한미약품',
 '현대제철', '휴젤', '금호석유', '포스코DX', 'CJ', '한온시스템', '엔켐', 'HPSP', 'JYP Ent.', '한전기술',
 '이수페타시스', '대한전선', '파마리서치', '신성델타테크', '롯데케미칼', '휠라홀딩스', 'F&F', '동서', '한화',
 '코오롱티슈진', '에스원', '펩트론', '산일전기', '셀트리온제약', '롯데지주', 'KCC', '아시아나항공', '한올바이오파마',
 '현대해상', '한화생명', '한전KPS', '농심', '현대엘리베이', '루닛', 'HL만도', 'SK가스', '더존비즈온', '에스엠',
 '아모레G', '제일기획', '이오테크닉스', '실리콘투', '한화엔진', '한미사이언스', '영원무역', '코스맥스', '테크윙',
 'CJ대한통운', '펄어비스', 'BGF리테일', '에이피알', '이마트', '팬오션', '에스티팜', '씨에스윈드', '일진전기', '한화비전',
 '파크시스템스', '코스모신소재', '에이비엘바이오', 'SK아이이테크놀로지', '녹십자', '대주전자재료', '코리안리', '한국앤컴퍼니',
 '대웅제약', '주성엔지니어링', '호텔신라', 'ISC', 'DGB금융지주', 'OCI홀딩스', '롯데쇼핑', '풍산', '오뚜기', 'GS건설',
 '케어젠', 'DB하이텍', 'HD현대인프라코어', '보로노이', '에스엘', '에스디바이오센서', 'LS에코에너지', '한국콜마',
 '대우건설', '서진시스템', '금호타이어', '솔브레인', '위메이드', '동원산업', '제이앤티씨', '하이트진로', '카카오게임즈',
 'HD현대건설기계', 'SK리츠', '경동나비엔', '신세계', 'HLB생명과학', 'GS리테일', '씨젠', 'DL이앤씨', '신영증권',
 '피에스케이홀딩스', 'DN오토모티브', '대웅', '동진쎄미켐', '브이티', '종근당', '네이처셀', 'CJ ENM', '원익IPS',
 '세방전지', '금양', '스튜디오드래곤', 'HDC현대산업개발', '현대백화점', '동원시스템즈', 'LG화학우', '영원무역홀딩스',
 '한샘', '두산퓨얼셀', '한솔케미칼', '현대위아', '고영', '더블유게임즈', '한일시멘트', 'DI동일', '롯데에너지머티리얼즈',
 '오스코텍', '이수스페셜티케미컬', '롯데정밀화학', 'HK이노엔', '롯데렌탈', '효성티앤씨', '대덕전자', 'ESR켄달스퀘어리츠',
 '제룡전기', '하이젠알앤엠', 'LX인터내셔널', '솔루엠', '롯데웰푸드', '메지온', '카페24', '롯데칠성', '와이씨', 'LX세미콘',
 'SK네트웍스', '미래에셋생명', '오리온홀딩스', '와이지엔터테인먼트', 'HD현대마린엔진', 'LS머트리얼즈', '피엔티', 'SOOP',
 '파라다이스', 'HLB테라퓨틱스', 'KB발해인프라', '유진테크', '프레스티지바이오파마', '국일제지', '보령', '롯데리츠',
 '티웨이항공', '세아제강지주', '메디톡스', '하나투어', '넥슨게임즈', '티씨케이', 'CJ CGV', '디어유', '삼성화재우',
 '태성', '성광벤드', '한국단자', '대신증권', 'HLB제약', '나노신소재', '미원상사', '율촌화학', '엠로', '쿠쿠홀딩스',
 '다우기술', 'HS효성첨단소재', '안랩', '중앙첨단소재', '레이크머티리얼즈']

base_keyword = kw_list[0]
iot = get_trends_data_in_batches(kw_list, base_keyword, pytrends)

# 최대값 계산 및 포맷팅
max_values = iot.max().sort_values(ascending=False)
max_values = max_values.round(2)

# 결과를 데이터프레임으로 변환하여 포맷팅
result_df = pd.DataFrame({
    '종목명': max_values.index,
    '관심도': max_values.values
})

# 출력 포맷 설정
pd.set_option('display.float_format', '{:.2f}'.format)
print("\n검색 관심도 분석 결과 (기준: 삼성전자)")
print("=" * 50)
print(f"{'종목명':<20} {'관심도':>10}")  # 헤더 포맷팅
print("-" * 50)
for _, row in result_df.iterrows():
    print(f"{row['종목명']:<20} {row['관심도']:>10.2f}")  # 데이터 포맷팅
print("=" * 50)


<ipython-input-123-97b9b110ed26>:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results[col] = data[col] / data[base_keyword] * 100
<ipython-input-123-97b9b110ed26>:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results[col] = data[col] / data[base_keyword] * 100
<ipython-input-123-97b9b110ed26>:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a d


검색 관심도 분석 결과 (기준: 삼성전자)
종목명                         관심도
--------------------------------------------------
NAVER                   1233.33
카카오                     1025.00
KT                       637.50
LG                       437.50
SOOP                     416.67
SK                       320.00
이마트                      241.94
기업은행                     235.90
기아                       200.00
신세계                      183.33
한화                       181.48
대한항공                     169.23
GS                       133.33
CJ                       123.08
삼성카드                     105.71
삼성전자                     100.00
카카오뱅크                    100.00
넷마블                      100.00
카카오페이                    100.00
SK하이닉스                    90.91
아시아나항공                    86.67
농심                        82.76
KCC                       79.31
파라다이스                     76.92
CJ CGV                    72.73
LG전자                      71.43
HD현대건설기계                  69.23
카페24                      69

<ipython-input-123-97b9b110ed26>:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results[base_keyword] = 100


In [81]:
from pytrends.request import TrendReq
from datetime import datetime, timedelta

# PyTrends 객체 초기화
pytrends = TrendReq(hl='ko-KR', tz=540)  # 한국어, 한국 시간대 설정

# 검색할 키워드 리스트
keywords = ["파이썬", "인공지능", "빅데이터", "블록체인", "메타버스"]
# keywords = ['삼성전자', 'SK하이닉스', 'LG에너지솔루션', '삼성바이오로직스', '현대차', '기아', '셀트리온', '삼성전자우', 'KB금융',]
#  'NAVER', 'HD현대중공업', '신한지주', '현대모비스', 'POSCO홀딩스', '메리츠금융지주', '삼성물산', '알테오젠', 'SK이노베이션',
#  '한화에어로스페이스', '고려아연', '한화오션', '하나금융지주', 'LG화학', '크래프톤', 'HMM', '삼성생명', '삼성화재',
#  'HD한국조선해양', 'HD현대일렉트릭', '카카오', '두산에너빌리티', '삼성SDI', 'LG전자', '한국전력', 'SK스퀘어', 'KT&G',
#  '에코프로비엠', '기업은행', 'KT', 'SK텔레콤', '우리금융지주', '한미반도체', 'LG', '삼성중공업', '포스코퓨처엠',
#  '현대글로비스', 'SK', '유한양행', 'HLB', '카카오뱅크', '삼성전기', '하이브', '대한항공', '삼성에스디에스', 'SK바이오팜',
#  '에코프로', 'HD현대마린솔루션', '포스코인터내셔널', '아모레퍼시픽', 'LS ELECTRIC', 'S-Oil', 'HD현대', 'DB손해보험',
#  '현대로템', 'SKC', '현대차2우B', '코웨이', '두산', '한진칼', '삼양식품', '두산밥캣', '한국항공우주', 'LIG넥스원',
#  '맥쿼리인프라', 'HD현대미포', '에코프로머티', '한화시스템', '한국타이어앤테크놀로지', '미래에셋증권', '레인보우로보틱스',
#  '삼성카드', '효성중공업', 'NH투자증권', 'LG생활건강', 'LG디스플레이', '리가켐바이오', 'LG유플러스', '한국금융지주',
#  '두산로보틱스', 'LS', '삼천당제약', '삼성증권', 'SK바이오사이언스', '오리온', 'BNK금융지주', '현대오토에버',
#  '엔씨소프트', 'JB금융지주', '현대차우', '넷마블', 'GS', 'CJ제일제당', '강원랜드', 'LG이노텍', '클래시스', '현대건설',
#  '삼성E&A', '카카오페이', '한화솔루션', '시프트업', '리노공업', '한국가스공사', '키움증권', '엘앤에프', '한미약품',
#  '현대제철', '휴젤', '금호석유', '포스코DX', 'CJ', '한온시스템', '엔켐', 'HPSP', 'JYP Ent.', '한전기술',
#  '이수페타시스', '대한전선', '파마리서치', '신성델타테크', '롯데케미칼', '휠라홀딩스', 'F&F', '동서', '한화',
#  '코오롱티슈진', '에스원', '펩트론', '산일전기', '셀트리온제약', '롯데지주', 'KCC', '아시아나항공', '한올바이오파마',
#  '현대해상', '한화생명', '한전KPS', '농심', '현대엘리베이', '루닛', 'HL만도', 'SK가스', '더존비즈온', '에스엠',
#  '아모레G', '제일기획', '이오테크닉스', '실리콘투', '한화엔진', '한미사이언스', '영원무역', '코스맥스', '테크윙',
#  'CJ대한통운', '펄어비스', 'BGF리테일', '에이피알', '이마트', '팬오션', '에스티팜', '씨에스윈드', '일진전기', '한화비전',
#  '파크시스템스', '코스모신소재', '에이비엘바이오', 'SK아이이테크놀로지', '녹십자', '대주전자재료', '코리안리', '한국앤컴퍼니',
#  '대웅제약', '주성엔지니어링', '호텔신라', 'ISC', 'DGB금융지주', 'OCI홀딩스', '롯데쇼핑', '풍산', '오뚜기', 'GS건설',
#  '케어젠', 'DB하이텍', 'HD현대인프라코어', '보로노이', '에스엘', '에스디바이오센서', 'LS에코에너지', '한국콜마',
#  '대우건설', '서진시스템', '금호타이어', '솔브레인', '위메이드', '동원산업', '제이앤티씨', '하이트진로', '카카오게임즈',
#  'HD현대건설기계', 'SK리츠', '경동나비엔', '신세계', 'HLB생명과학', 'GS리테일', '씨젠', 'DL이앤씨', '신영증권',
#  '피에스케이홀딩스', 'DN오토모티브', '대웅', '동진쎄미켐', '브이티', '종근당', '네이처셀', 'CJ ENM', '원익IPS',
#  '세방전지', '금양', '스튜디오드래곤', 'HDC현대산업개발', '현대백화점', '동원시스템즈', 'LG화학우', '영원무역홀딩스',
#  '한샘', '두산퓨얼셀', '한솔케미칼', '현대위아', '고영', '더블유게임즈', '한일시멘트', 'DI동일', '롯데에너지머티리얼즈',
#  '오스코텍', '이수스페셜티케미컬', '롯데정밀화학', 'HK이노엔', '롯데렌탈', '효성티앤씨', '대덕전자', 'ESR켄달스퀘어리츠',
#  '제룡전기', '하이젠알앤엠', 'LX인터내셔널', '솔루엠', '롯데웰푸드', '메지온', '카페24', '롯데칠성', '와이씨', 'LX세미콘',
#  'SK네트웍스', '미래에셋생명', '오리온홀딩스', '와이지엔터테인먼트', 'HD현대마린엔진', 'LS머트리얼즈', '피엔티', 'SOOP',
#  '파라다이스', 'HLB테라퓨틱스', 'KB발해인프라', '유진테크', '프레스티지바이오파마', '국일제지', '보령', '롯데리츠',
#  '티웨이항공', '세아제강지주', '메디톡스', '하나투어', '넥슨게임즈', '티씨케이', 'CJ CGV', '디어유', '삼성화재우',
#  '태성', '성광벤드', '한국단자', '대신증권', 'HLB제약', '나노신소재', '미원상사', '율촌화학', '엠로', '쿠쿠홀딩스',
#  '다우기술', 'HS효성첨단소재', '안랩', '중앙첨단소재', '레이크머티리얼즈']

# 날짜 설정 (최근 30일)
end_date = datetime.now() - timedelta(days=1)  # 어제
start_date = end_date - timedelta(days=30)  # 30일 전
timeframe = f'{start_date.strftime("%Y-%m-%d")} {end_date.strftime("%Y-%m-%d")}'

# 트렌드 데이터 요청 (5개씩 나누어서 요청)
for i in range(0, len(keywords), 5):
    keyword_chunk = keywords[i:i + 5]
    pytrends.build_payload(keyword_chunk, cat=0, timeframe=timeframe, geo='KR')
    related_queries = pytrends.related_queries()

    # 상승 키워드 추출 및 정렬
    rising_keywords = []
    for keyword in keyword_chunk:
        if keyword in related_queries:
            query_data = related_queries[keyword]
            if query_data and 'rising' in query_data and query_data['rising'] is not None:
                rising_df = query_data['rising']
                if not rising_df.empty:
                    rising_keywords.extend(rising_df['query'].tolist())

    # 중복 제거 및 상위 10개 선택
    if rising_keywords:
        top_10_rising = list(dict.fromkeys(rising_keywords))[:10]

        print(f"\n키워드 {keyword_chunk}에 대한 최근 상승 키워드 Top 10:")
        for j, kw in enumerate(top_10_rising, 1):
            print(f"{j}. {kw}")
    else:
        print(f"\n키워드 {keyword_chunk}에 대한 상승 키워드가 없습니다.")



IndexError: list index out of range

In [90]:
from pytrends.request import TrendReq
from datetime import datetime, timedelta
import time
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def create_custom_session():
    """Create a custom session with retry strategy"""
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "POST", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    return session

def get_trends_data(keywords, country='KR', days=30, debug=True):
    """
    Enhanced Google Trends data fetching with multiple data points
    """
    try:
        if debug:
            print("Initializing PyTrends with custom session...")

        session = create_custom_session()
        pytrends = TrendReq(
            hl=f'{country.lower()}-{country.upper()}',
            tz=540,
            timeout=(10,25),
            requests_args={'verify': True}
        )
        pytrends.requests = session

        # Calculate date range
        end_date = datetime.now() - timedelta(days=1)
        start_date = end_date - timedelta(days=days)
        timeframe = f'{start_date.strftime("%Y-%m-%d")} {end_date.strftime("%Y-%m-%d")}'

        if debug:
            print(f"Timeframe set to: {timeframe}")

        results = {}

        for keyword in keywords:
            if debug:
                print(f"\nAnalyzing keyword: {keyword}")

            try:
                # Add delay between requests
                time.sleep(3)

                if debug:
                    print(f"Building payload for: {keyword}")

                # First check interest over time
                pytrends.build_payload(
                    kw_list=[keyword],
                    cat=0,
                    timeframe=timeframe,
                    geo=country
                )

                # Get interest over time
                interest_df = pytrends.interest_over_time()

                if not interest_df.empty:
                    results[keyword] = {
                        'interest_over_time': interest_df[keyword].to_dict()
                    }

                    if debug:
                        print(f"Found interest data for '{keyword}'")
                        print(f"Average interest: {interest_df[keyword].mean():.2f}")

                    # Try to get related topics
                    try:
                        related_topics = pytrends.related_topics()
                        if related_topics and keyword in related_topics:
                            topic_data = related_topics[keyword]
                            if 'rising' in topic_data and topic_data['rising'] is not None:
                                rising_topics = topic_data['rising']
                                if not rising_topics.empty:
                                    print(f"\nRising related topics for '{keyword}':")
                                    print(rising_topics[['topic_title', 'value']].head(10))
                    except Exception as e:
                        if debug:
                            print(f"Could not fetch related topics: {str(e)}")

                    # Try suggestions API
                    try:
                        suggestions = pytrends.suggestions(keyword)
                        if suggestions:
                            print(f"\nSearch suggestions for '{keyword}':")
                            for sugg in suggestions[:10]:
                                print(f"- {sugg['title']}")
                    except Exception as e:
                        if debug:
                            print(f"Could not fetch suggestions: {str(e)}")

                    # Get interest by region
                    try:
                        region_interest = pytrends.interest_by_region(resolution='COUNTRY')
                        if not region_interest.empty:
                            region_data = region_interest[region_interest[keyword] > 0].sort_values(by=keyword, ascending=False)
                            if not region_data.empty:
                                print(f"\nTop countries interested in '{keyword}':")
                                print(region_data.head(5))
                    except Exception as e:
                        if debug:
                            print(f"Could not fetch regional interest: {str(e)}")

                else:
                    print(f"\nNo trend data found for '{keyword}'")

            except Exception as e:
                print(f"Error processing keyword '{keyword}': {str(e)}")
                if debug:
                    import traceback
                    print(f"Traceback: {traceback.format_exc()}")
                continue

            time.sleep(2)

        return results

    except Exception as e:
        print(f"Fatal error: {str(e)}")
        if debug:
            import traceback
            print(f"Traceback: {traceback.format_exc()}")
        return None

if __name__ == "__main__":
    test_keywords = ["파이썬", "인공지능", "빅데이터", "블록체인", "메타버스"]
    test_keywords = ['삼성전자', 'SK하이닉스', 'LG에너지솔루션', '삼성바이오로직스', '현대차', '기아', '셀트리온', '삼성전자우', 'KB금융',
    'NAVER', 'HD현대중공업', '신한지주', '현대모비스', 'POSCO홀딩스', '메리츠금융지주', '삼성물산', '알테오젠', 'SK이노베이션',
    '한화에어로스페이스', '고려아연', '한화오션', '하나금융지주', 'LG화학', '크래프톤', 'HMM', '삼성생명', '삼성화재',
    'HD한국조선해양', 'HD현대일렉트릭', '카카오', '두산에너빌리티', '삼성SDI', 'LG전자', '한국전력', 'SK스퀘어', 'KT&G',
    '에코프로비엠', '기업은행', 'KT', 'SK텔레콤', '우리금융지주', '한미반도체', 'LG', '삼성중공업', '포스코퓨처엠',
    '현대글로비스', 'SK', '유한양행', 'HLB', '카카오뱅크', '삼성전기', '하이브', '대한항공', '삼성에스디에스', 'SK바이오팜',
    '에코프로', 'HD현대마린솔루션', '포스코인터내셔널', '아모레퍼시픽', 'LS ELECTRIC', 'S-Oil', 'HD현대', 'DB손해보험',
    '현대로템', 'SKC', '현대차2우B', '코웨이', '두산', '한진칼', '삼양식품', '두산밥캣', '한국항공우주', 'LIG넥스원',
    '맥쿼리인프라', 'HD현대미포', '에코프로머티', '한화시스템', '한국타이어앤테크놀로지', '미래에셋증권', '레인보우로보틱스',
    '삼성카드', '효성중공업', 'NH투자증권', 'LG생활건강', 'LG디스플레이', '리가켐바이오', 'LG유플러스', '한국금융지주',
    '두산로보틱스', 'LS', '삼천당제약', '삼성증권', 'SK바이오사이언스', '오리온', 'BNK금융지주', '현대오토에버',
    '엔씨소프트', 'JB금융지주', '현대차우', '넷마블', 'GS', 'CJ제일제당', '강원랜드', 'LG이노텍', '클래시스', '현대건설',
    '삼성E&A', '카카오페이', '한화솔루션', '시프트업', '리노공업', '한국가스공사', '키움증권', '엘앤에프', '한미약품',
    '현대제철', '휴젤', '금호석유', '포스코DX', 'CJ', '한온시스템', '엔켐', 'HPSP', 'JYP Ent.', '한전기술',
    '이수페타시스', '대한전선', '파마리서치', '신성델타테크', '롯데케미칼', '휠라홀딩스', 'F&F', '동서', '한화',
    '코오롱티슈진', '에스원', '펩트론', '산일전기', '셀트리온제약', '롯데지주', 'KCC', '아시아나항공', '한올바이오파마',
    '현대해상', '한화생명', '한전KPS', '농심', '현대엘리베이', '루닛', 'HL만도', 'SK가스', '더존비즈온', '에스엠',
    '아모레G', '제일기획', '이오테크닉스', '실리콘투', '한화엔진', '한미사이언스', '영원무역', '코스맥스', '테크윙',
    'CJ대한통운', '펄어비스', 'BGF리테일', '에이피알', '이마트', '팬오션', '에스티팜', '씨에스윈드', '일진전기', '한화비전',
    '파크시스템스', '코스모신소재', '에이비엘바이오', 'SK아이이테크놀로지', '녹십자', '대주전자재료', '코리안리', '한국앤컴퍼니',
    '대웅제약', '주성엔지니어링', '호텔신라', 'ISC', 'DGB금융지주', 'OCI홀딩스', '롯데쇼핑', '풍산', '오뚜기', 'GS건설',
    '케어젠', 'DB하이텍', 'HD현대인프라코어', '보로노이', '에스엘', '에스디바이오센서', 'LS에코에너지', '한국콜마',
    '대우건설', '서진시스템', '금호타이어', '솔브레인', '위메이드', '동원산업', '제이앤티씨', '하이트진로', '카카오게임즈',
    'HD현대건설기계', 'SK리츠', '경동나비엔', '신세계', 'HLB생명과학', 'GS리테일', '씨젠', 'DL이앤씨', '신영증권',
    '피에스케이홀딩스', 'DN오토모티브', '대웅', '동진쎄미켐', '브이티', '종근당', '네이처셀', 'CJ ENM', '원익IPS',
    '세방전지', '금양', '스튜디오드래곤', 'HDC현대산업개발', '현대백화점', '동원시스템즈', 'LG화학우', '영원무역홀딩스',
    '한샘', '두산퓨얼셀', '한솔케미칼', '현대위아', '고영', '더블유게임즈', '한일시멘트', 'DI동일', '롯데에너지머티리얼즈',
    '오스코텍', '이수스페셜티케미컬', '롯데정밀화학', 'HK이노엔', '롯데렌탈', '효성티앤씨', '대덕전자', 'ESR켄달스퀘어리츠',
    '제룡전기', '하이젠알앤엠', 'LX인터내셔널', '솔루엠', '롯데웰푸드', '메지온', '카페24', '롯데칠성', '와이씨', 'LX세미콘',
    'SK네트웍스', '미래에셋생명', '오리온홀딩스', '와이지엔터테인먼트', 'HD현대마린엔진', 'LS머트리얼즈', '피엔티', 'SOOP',
    '파라다이스', 'HLB테라퓨틱스', 'KB발해인프라', '유진테크', '프레스티지바이오파마', '국일제지', '보령', '롯데리츠',
    '티웨이항공', '세아제강지주', '메디톡스', '하나투어', '넥슨게임즈', '티씨케이', 'CJ CGV', '디어유', '삼성화재우',
    '태성', '성광벤드', '한국단자', '대신증권', 'HLB제약', '나노신소재', '미원상사', '율촌화학', '엠로', '쿠쿠홀딩스',
    '다우기술', 'HS효성첨단소재', '안랩', '중앙첨단소재', '레이크머티리얼즈']

    results = get_trends_data(test_keywords, debug=True)

Initializing PyTrends with custom session...
Timeframe set to: 2024-12-29 2025-01-28

Analyzing keyword: 삼성전자
Building payload for: 삼성전자
Found interest data for '삼성전자'
Average interest: 58.84
Could not fetch related topics: list index out of range

Search suggestions for '삼성전자':
- Samsung Electronics
- Samsung Galaxy Watch6 Classic
- Samsung Galaxy Tab S2 9.7
- Samsung Electronics America
- SamsungAdvancedInstituteofTechnology

Top countries interested in '삼성전자':
                   삼성전자
geoName                
Gyeongsangbuk-do    100
Gyeonggi-do          91
Ulsan                84
Chungcheongnam-do    83
Daejeon              82

Analyzing keyword: SK하이닉스
Building payload for: SK하이닉스
Found interest data for 'SK하이닉스'
Average interest: 59.32
Could not fetch related topics: list index out of range

Search suggestions for 'SK하이닉스':
- SK Hynix
- SK hynix America
- SK hynix memory solutions America Inc.
- SK Hynix Bundang office
- SK Hynix Dormitory

Top countries interested in 'SK하이닉스':
    

KeyboardInterrupt: 